1. 원하는 데이터가 있는 사이트를 찾았는가? ok
2. 페이지 소스 보기를 하였을 때 페이지 소스에 원하는 데이터가 있는가? ok
3. 주소 분석이 가능한가? ok
4. 끝나는 시점을 알 수 있는가? ok

-> BS4를 사용하는 것을 권장드립니다. 

### BS4의 특정
- 웹 요청 주소를 이용해 서버에 직접 요청을 할 수 있다.
- 내부적으로 웹 서버가 보내주는 데이터를 받아다 사용하기 때문에 속도가 selenium에 비해서 빠르다.
- 하지만 DOM 코드를 가져다 사용할 수는 없다.
- 웹 문서에서 소스보기를 했을 때 보이는 코드(즉 서버에서 받아온 코드)를 분석할 때 사용한다.

### BS4를 통해 데이터를 수집하는 코드 작성 순서
1. 첫 번째 페이지에서 원하는 데이터를 가져올 수 있도록 구현한다.
2. 다음 페이지로 넘어갈 수 있도록 구현한다.
3. 마지막 페이지 여부를 확인하여 중단시킬 수 있도록 구현한다.
4. 모든 구현이 끝나면 몇 페이지 돌려보세요.
5. 반드시 요청하는 페이지의 주소를 파일에 기록하세요!

In [1]:
# 가져온 html 데이터 분석을 위한 라이브러리
import bs4
# 웹 서버에 요청을 하기 위한 라이브러리
import requests
# 데이터 분석 라이브러리; 수집한 데이터를 저장하기 위해 사용하겠습니다
import pandas as pd
# 행렬 및 선형대수학을 위한 라이브러리; 결측치 값을 사용하기 위해 사용
import numpy as np
# 딜레이를 위해..
import time
# 컴퓨터나 os와 관련된 라이브러리; 파일 존재 여부를 확인하기 위해 사용
import os
# 주피터 노트북에서 출력된 내용을 지우기 위해 사용함
from IPython.display import clear_output

In [2]:
# 요청함수
def getSource(site) :
    """
        특정 웹 사이트에 접속하여 bs4 객체를 생성해 반환한다.

        site 
            요청할 페이지의 주소

        return 값
            html 소스가 담겨져 있는 bs4 객체
    """

    # 해더 정보 설정
    # user-agent : 웹 브라우저가 서버로 보내는 문자열이고 서버는 이를 통해 브라우저 정보나 컴퓨터 정보를 파악한다.
    # 일부 사이트에는 user-agent가 전달되지 않으면 데이터를 전달하지 않는 경우도 있다.
    # 이에 user-agent를 셋팅하여 요청한 도구가 python 코드를 통한 것이 아닌 일반 웹브라우저를 통해 요청한 것처럼 속일 수 있다.
    # https://m.avalon.co.kr/check.html 에서 user-agent 확인이 가능함. 
    header_info = {
        'User-Agent' : 'Mozilla/5.0(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36'
    }
    # 요청한다.
    response = requests.get(site, headers=header_info)
    # bs4 객체를 생성한다
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    return soup

In [3]:
# 한 페이지의 데이터를 수집해 저장하는 함수
def getData(soup, file_name) : 
    """
        한 페이지의 데이터를 수집해 파일에 저장하는 함수

        soup
            HTML 데이터를 관리하는 bs4 객체

        file_name
            데이터를 저장할 파일 이름
    """

    # 데이터가 있는 전체를 가져온다.
    a1 = soup.select_one('body > main > div > div')
    # print(a1)

    # 수집한 데이터를 담아 저장하는 용도로 사용할
    # 데이터 프레임을 생성하기 위해 사용할 딕셔너리
    data_dict = {
        '큰제목' : [],
        '작은제목' : [],
        '날짜' : [],
        '작성자' : [],
    }

    # 내부의 div 태그들을 가져온다. 
    a2 = a1.select('div')[:-1]
    for div in a2 :
       # print(div)
        # 큰 제목을 가져온다.
        a3 = div.select_one('h3 > a')
        data1 = a3.text.strip()
        if len(data1) == 0:
            data1 = np.nan #길이가 0이라면 결측치에 해당하는 값을 가져온다
        # print(f'data1 : {data1}')
        # print('----------------------')
        
        # 작은 제목을 가져온다.
        a4 = div.select_one('h4')
        data2 = a4.text.strip()
        if len(data2) == 0:
            data2 = np.nan
        # print(f'data2 : {data2}')
        # print('----------------------')

        # 날짜와 작성자
        a5 = div.select_one('p > span')
        a6 = a5.text.split('|')
        data3 = a6[0].strip()
        if len(data3) == 0:
            data3 = np.nan
        data4 = a6[1].strip()
        if len(data4) == 0:
            data4 = np.nan
        print(f'data3 : {data3}')
        print(f'data4 : {data4}')

        # 딕셔너리에 데이터를 담는다.
        data_dict['큰제목'].append(data1)
        data_dict['작은제목'].append(data2)
        data_dict['날짜'].append(data3)
        data_dict['작성자'].append(data4)

    # 데이터 프레임을 생성한다.
    df1 = pd.DataFrame(data_dict)
    # display(df1)

    # 저장한다.
    # 만약 파일이 없다면
    if os.path.exists(file_name) == False :
        df1.to_csv(file_name, encoding='utf-8-sig', index=False)

    # 만약 파일이 있다면
    else : 
        df1.to_csv(file_name, encoding='utf-8-sig', index=False, header=None, mode='a')
    

In [4]:
# 다음 페이지 주소를 가져오는 함수
def getNextPage(soup) :
    """
        다음 페이지 주소를 가져오는 함수

        soup 
            HTML 데이터를 가지고 있는  bs4 객체

        반환값
            다음 페이지가 없다면 None을 반환한다.
            다음 페이지가 있다면 요청할 주소에 붙힐 다음 페이지에 대한 정보를 반환한다
    """
    # Next 버튼의 태그를 가져온다.
    next_tag = soup.select_one('body > main > div > div > div.pagination > ul > li:nth-child(6) > a')
    # print(next_tag)    
    # 가져온 것이 있다면
    if next_tag != None :
        # href 속성의 값을 가져온다.
        href = next_tag.attrs['href']
        return href
    else :
        return None

In [5]:
# 요청할 페이지의 주소 (첫 페이지 주소)
# 어떤 페이지인지를 나타내는 값이 있으면 지워주세요
site = 'https://pjt3591oo.github.io/'
# 수집을 하고자 하는 페이지를 나타내는 값
page = ''

# 반복한다.
while True : 
    # 딜레이
    time.sleep(1)

    # 기존에 출력된 것을 청소한다.
    clear_output(wait = True)

    print(f'{site}{page} 수집중.. ')
    # 현재 수집하고자 하는 사이트의 주소를 파일에 기록한다.
    # r : 읽기, w : 기존 내용 지우고 쓰기, a : 기존 내용에 이어서 쓰기
    with open('02_log.txt', 'a', encoding='utf-8') as fp : 
        fp.write(f'{site}{page}\n')

    # 페이지 요청
    soup = getSource(site + page)
    # 현재 페이지에서 데이터를 가져와 저장한다.
    getData(soup, 'data1.csv')
    # 다음 페이지의 정보를 가져온다.
    page = getNextPage(soup)

    # 만약 다음 페이지가 없다면 중단한다.
    if page == None :
        print('수집완료')
        break


#soup = getSource('https://pjt3591oo.github.io/page4/')
#print(getNextPage(soup))

https://pjt3591oo.github.io//page4/ 수집중.. 
data3 : 2017-04-04 13:10:05 +0000
data4 : 박정태
수집완료
